In [28]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium

## Next time - need to fix MTA data to make it geodata, and then plot MTA stops in Antioch with schools in Antioch.

In [29]:
zipcode = gpd.read_file('/Users/rabram/Desktop/NSS/Nashville-Public-Art/data/Zip Codes (GIS).geojson')

mta = pd.read_csv('/Users/rabram/Desktop/NSS/Nashville-Public-Art/data/Metropolitan Transit Authority (MTA) Bus Routes (GIS).geojson')

ParserError: Error tokenizing data. C error: Expected 2 fields in line 4, saw 216764


In [3]:
zipcode.head()

,zip,objectid,po_name,shape_stlength,shape_starea,geometry
0,37027,34,BRENTWOOD,3140.581074908484,595244.46435546875,"(POLYGON ((-86.684745973484 35.992792089723, -..."
1,37027,37,BRENTWOOD,3558.4949049555025,315672.26501464844,"(POLYGON ((-86.719974241412 36.009524281615, -..."
2,37143,29,PEGRAM,7053.537934957626,1901096.087890625,"(POLYGON ((-87.03537311557 36.087623620297, -8..."
3,37143,28,PEGRAM,1197.1070870572601,83126.842468261719,"(POLYGON ((-87.03605109734001 36.095325458066,..."
4,37240,44,NASHVILLE,9240.7969963306841,3146698.6988525391,"(POLYGON ((-86.803254434485 36.149298486807, -..."


In [4]:
mta.head()

,line_name,line_abbr,agency_id,agency_id_1,line_id,agency_id_0,geometry
0,100 OAKS,1,1,1,1841,1,"(LINESTRING (-86.781413 36.166762, -86.781668 ..."
1,BELMONT,2,1,1,1848,1,"(LINESTRING (-86.781413 36.166762, -86.781668 ..."
2,WEST END - WHITE BRIDGE,3,1,1,1905,1,"(LINESTRING (-86.808134 36.147076, -86.808342 ..."
3,SHELBY,4,1,1,1866,1,"(LINESTRING (-86.781413 36.166762, -86.781668 ..."
4,WEST END - BELLEVUE,5,1,1,1873,1,"(LINESTRING (-86.808134 36.147076, -86.808342 ..."


In [5]:
mta_zip = gpd.sjoin(mta, zipcode)

In [6]:
mta_zip.head()

,line_name,line_abbr,agency_id,agency_id_1,line_id,agency_id_0,geometry,index_right,zip,objectid,po_name,shape_stlength,shape_starea
0,100 OAKS,1,1,1,1841,1,"(LINESTRING (-86.781413 36.166762, -86.781668 ...",25,37203,40,NASHVILLE,92627.871628329856,121775086.45056152
1,BELMONT,2,1,1,1848,1,"(LINESTRING (-86.781413 36.166762, -86.781668 ...",25,37203,40,NASHVILLE,92627.871628329856,121775086.45056152
2,WEST END - WHITE BRIDGE,3,1,1,1905,1,"(LINESTRING (-86.808134 36.147076, -86.808342 ...",25,37203,40,NASHVILLE,92627.871628329856,121775086.45056152
4,WEST END - BELLEVUE,5,1,1,1873,1,"(LINESTRING (-86.808134 36.147076, -86.808342 ...",25,37203,40,NASHVILLE,92627.871628329856,121775086.45056152
5,LEBANON PIKE,6,1,1,1878,1,"(LINESTRING (-86.781413 36.166762, -86.781668 ...",25,37203,40,NASHVILLE,92627.871628329856,121775086.45056152


In [7]:
mta_zip[['zip', 'line_id']].groupby('zip').agg('count').sort_values(by ='line_id', ascending = False)

,line_id
zip,
37219,44
37201,34
37203,34
37210,20
37213,18
37206,16
37208,13
37211,11
37212,10


In [8]:
schools = pd.read_csv('/Users/rabram/Desktop/NSS/Nashville-Public-Art/data/Metro_Nashville_Public_School_Locations.csv')

In [9]:
schools.columns = ['SID','name','address','city','state','zip','phone','SSID','type','lowest_grade','highest_grade','lat','lng','website',"cluster","coordinates"]

In [10]:
schools['geometry'] = schools.apply(lambda x: Point((float(x.lng), float(x.lat))), axis=1)

In [11]:
schools_geo = gpd.GeoDataFrame(schools, crs = zipcode.crs, geometry = schools['geometry'])

In [12]:
schools_geo[['zip', 'name']].groupby('zip').agg('count').sort_values(by ='name', ascending = False)

,name
zip,
37013,40
37207,32
37211,30
37209,30
37206,24
37214,18
37210,18
37216,16
37208,16


In [13]:
schools37013 = schools_geo.loc[schools_geo['zip'] == 37013]

In [14]:
mta37013 = mta_zip.loc[mta_zip['zip'] == 37013]

In [15]:
mta37013.head()

,line_name,line_abbr,agency_id,agency_id_1,line_id,agency_id_0,geometry,index_right,zip,objectid,po_name,shape_stlength,shape_starea


In [16]:
zip_poly = zipcode.loc[zipcode.zip == '37013']

In [17]:
zip_poly.geometry.centroid

42    POINT (-86.63636274154366 36.04173095575453)
dtype: object

In [18]:
center37013 = [36.04173095575453, -86.63636274154366]